<a href="https://colab.research.google.com/github/Ankit-03G/Stealth_1/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting the Stage for Object Detection

Our journey begins by preparing our environment for the fascinating task of object detection using the custom YOLOv11 model.

First, we ensure all the necessary tools are at our fingertips by installing the required libraries.

*   **`ultralytics`**: The heart of our operation, providing the powerful YOLO framework.
*   **`torch`, `torchvision`, `torchaudio`**: The robust PyTorch deep learning framework, enabling efficient computation and image handling.
*   **`opencv-python`**: Our go-to for all things related to computer vision, perfect for manipulating and visualizing images.
*   **`matplotlib`** and **`seaborn`**: For creating insightful plots and understanding our model's performance.
*   **`pillow`**: A fundamental library for seamless image processing.

With our tools in place, we set up a system to keep track of our progress and any important messages during the process – this is our logging system, ensuring we don't miss a beat.



In [ ]:
!pip install ultralytics
!pip install torch torchvision torchaudio
!pip install opencv-python
!pip install matplotlib seaborn
!pip install pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import yaml
import os
import shutil
from pathlib import Path
import cv2
import numpy as np
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors
import matplotlib.pyplot as plt
from google.colab import drive
import logging

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Setting up Basic logging


In [ ]:

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Downloading the provided model

Next, we bring in the key players: our pre-trained YOLOv11 model provided to us earlier in the assignment a powerful foundation for our task, and the dataset we'll use to train and refine this model to recognize specific objects.

In [ ]:
# Download the model file
model_file_id = '1OGOv8oFxdDWLfxSa3ZqdK_IdleqQQJtm'
model_destination = 'your_model.pt'
!gdown --id {model_file_id} -O {model_destination}

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1OGOv8oFxdDWLfxSa3ZqdK_IdleqQQJtm
From (redirected): https://drive.google.com/uc?id=1OGOv8oFxdDWLfxSa3ZqdK_IdleqQQJtm&confirm=t&uuid=2e4f8639-c72d-4271-9701-3b4c9367412b
To: /content/your_model.pt
100% 195M/195M [00:02<00:00, 86.8MB/s]


## About the Dataset

This dataset has been **prepared and labelled by me from start to end** and was not provided as a part of the project .

## Steps Involved

1. I fed the two videos(broadcast and tacticam) in a python script that will made frames out of the video (132 in case of broadcast and 201 in case of tacticam)

2. I then selected 10 frames from each covering the whole clips

3. I then downloaded the frames as images with proper names

4. I then processed the images in label-studio by setting up classes from Player 1-22 , reffree and ball

5. I then **manually labelled each player consistently with the same Player id** throughout all the images . Example - Labelled Goalkeeper as Player 12 Throughout all the images

6. After labelling everything , downloaded the dataset as zip and here i am fetching the database

In [ ]:
# Download the dataset folder
dataset_folder_id = '1hjH1LtQPFBXz303fGgz89Oxexk2iyBJj'
dataset_destination = 'dataset'
!gdown --folder --id {dataset_folder_id} -O {dataset_destination}

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Retrieving folder contents
Processing file 1w6J7IcN3356mrSjlhjnGpRrWM4gn5coX classes.txt
Processing file 109dSTvqQr9tQnQNdY-lQGch-BlWT_jMb notes.json
Retrieving folder 1q2wJea9uSx6Ldtdlq8XFGDbEoII3Cz55 images
Processing file 1KYnAXU4UVfO2hCUiPpwIUxpBQpQETE7E 7d2d5eea-broadcast_frame_0118.jpg
Processing file 1teE5IzDzcT2-5ouI2ID4l1lEpBPnUo-O 69a0fe66-broadcast_frame_0079.jpg
Processing file 1l6Vkmg6Rwvl6GeOPZHeIZeN-Thmdxqeo 4dd387e2-broadcast_frame_0013.jpg
Processing file 1wdPZrVAqNMFOqzTptPexecmhGTqj2Zcr 52e77c19-broadcast_frame_0026.jpg
Processing file 1c2q_af15jjmY8wNOtAKq9XjeiNZhd5bX 9fc2434b-broadcast_frame_0052.jpg
Processing file 1lUMiT5DmukiXdPjco-gZwnkfrIenIszP b26a0ba5-tacticam_frame_0140.jpg
Processing file 17VQ2DQnC7bBnoa5vMO1VN3Zf_TrYhvDx 0ced6b47-tacticam

## Creating a .yaml file

Finally, we configure the YOLOv11 model with a vital piece of information: the details of our dataset, including the classes of objects it needs to identify. This is done by creating a YAML configuration file, a clear instruction manual for the model's training process.

In [ ]:
import yaml
import os

# Define paths
dataset_dir = '/content/dataset'
classes_file = os.path.join(dataset_dir, 'classes.txt')
yaml_file_path = '/content/yolov11_config.yaml'

# Read class names from classes.txt
with open(classes_file, 'r') as f:
    class_names = f.read().splitlines()

# Create the YAML configuration
data = {
    'path': dataset_dir,
    'train': 'images',  # Assuming images are in a folder named 'images' inside the dataset directory
    'val': 'images',    # Assuming validation images are also in 'images' or specify a different folder if you have one
    'nc': len(class_names),
    'names': class_names
}

# Write the YAML file
with open(yaml_file_path, 'w') as f:
    yaml.dump(data, f, default_flow_style=None)

print(f"YOLOv11 configuration file created at: {yaml_file_path}")

YOLOv11 configuration file created at: /content/yolov11_config.yaml


## Training the model for the first time

 I trained the model on the dataset with a batch size of 8

 Got the following results :-  

mAP50: 0.845      
mAP50-95: 0.465     


These metrics indicate how well your model performed in detecting objects in the validation set after the training. A higher value for both mAP50 and mAP50-95 indicates better performance.

In [ ]:
from ultralytics import YOLO

# Load the YOLOv11 model
model = YOLO(MODEL_PATH)

# Start training
results = model.train(data=yaml_file_path, epochs=100, imgsz=640, batch=8) # You can adjust epochs and imgsz as needed

Ultralytics 8.3.164 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolov11_config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/your_model.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0

100%|██████████| 755k/755k [00:00<00:00, 22.3MB/s]

Overriding model.yaml nc=4 with nc=24

                   from  n    params  module                                       arguments                     
  0                  -1  1      8800  ultralytics.nn.modules.conv.Conv             [3, 80, 6, 2, 2]              
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  4    309120  ultralytics.nn.modules.block.C3              [160, 160, 4]                 
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  8   2259200  ultralytics.nn.modules.block.C3              [320, 320, 8]                 
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              


  6                  -1 12  13125120  ultralytics.nn.modules.block.C3              [640, 640, 12]                
  7                  -1  1   7375360  ultralytics.nn.modules.conv.Conv             [640, 1280, 3, 2]             
  8                  -1  4  19676160  ultralytics.nn.modules.block.C3              [1280, 1280, 4]               
  9                  -1  1   4099840  ultralytics.nn.modules.block.SPPF            [1280, 1280, 5]               
 10                  -1  1    820480  ultralytics.nn.modules.conv.Conv             [1280, 640, 1, 1]             
 11                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 12             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 13                  -1  4   5332480  ultralytics.nn.modules.block.C3              [1280, 640, 4, False]         
 14                  -1  1    205440  ultralytics.nn.modules.conv.Conv             [640,

100%|██████████| 5.35M/5.35M [00:00<00:00, 101MB/s]


...

    100/100      8.98G        1.5      1.315     0.9621         76        640: 100%|██████████| 3/3 [00:01<00:00,  1.68it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.08it/s]

                   all         20        333      0.728      0.781      0.846      0.465



100 epochs completed in 0.312 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 195.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 195.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.164 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv5x summary (fused): 150 layers, 97,175,720 parameters, 0 gradients, 246.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.61it/s]


                   all         20        333      0.731      0.782      0.845      0.465
                  ball         15         15          1      0.516        0.6      0.304
             player: 1         20         20       0.59        0.8      0.777      0.359
            player: 10         10         10      0.535          1      0.995      0.549
            player: 11         10         10      0.665          1      0.995       0.52
            player: 12         15         15        0.9          1      0.995      0.652
            player: 13         19         19      0.825      0.789      0.898      0.616
            player: 14         19         19      0.644          1       0.98      0.647
            player: 15         19         19      0.744      0.789      0.805        0.4
            player: 16         19         19       0.73      0.895      0.894      0.475
            player: 17         19         19      0.482      0.895      0.865      0.466
            player: 1

## Training the model for the 2nd time :-  

Trained the model again which took around 1.5 hours and got these results :-  

mAP50: 0.875   
mAP50-95: 0.495

In [ ]:
from ultralytics import YOLO

# Load the YOLOv11 model - this will likely load the last saved checkpoint (best.pt or last.pt)
model = YOLO(MODEL_PATH)

# Start training again - it will resume from the last checkpoint
# You can adjust the number of epochs if you want to train for more than another 100
results = model.train(data=yaml_file_path, epochs=100, imgsz=640, batch=8)

Ultralytics 8.3.164 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolov11_config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/your_model.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.

train: Scanning /content/dataset/labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1186.2±1102.0 MB/s, size: 542.1 KB)


val: Scanning /content/dataset/labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]


...

    100/100       8.7G        1.5      1.315     0.9621         76        640: 100%|██████████| 3/3 [00:01<00:00,  1.80it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.54it/s]

                   all         20        333      0.728      0.781      0.846      0.465



100 epochs completed in 1.862 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 195.0MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 195.0MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.164 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv5x summary (fused): 150 layers, 97,175,720 parameters, 0 gradients, 246.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.85it/s]


                   all         20        333      0.731      0.782      0.845      0.465
                  ball         15         15          1      0.516        0.6      0.304
             player: 1         20         20       0.59        0.8      0.777      0.359
            player: 10         10         10      0.535          1      0.995      0.549
            player: 11         10         10      0.665          1      0.995       0.52
            player: 12         15         15        0.9          1      0.995      0.652
            player: 13         19         19      0.825      0.789      0.898      0.616
            player: 14         19         19      0.644          1       0.98      0.647
            player: 15         19         19      0.744      0.789      0.805        0.4
            player: 16         19         19       0.73      0.895      0.894      0.475
            player: 17         19         19      0.482      0.895      0.865      0.466
            player: 1

## After this we move to the next cell as the We exceeded the Computation Limits